In [8]:
from models.rnn.rnn_plain import RNN_Plain
from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [10]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2500, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)

In [14]:
algo.train(train, input_lang, output_lang)

0m 26s (- 3m 55s) (250 10%) 1.5326
0m 52s (- 3m 31s) (500 20%) 1.5043
1m 13s (- 2m 51s) (750 30%) 1.5223
1m 39s (- 2m 29s) (1000 40%) 1.5135
1m 56s (- 1m 56s) (1250 50%) 1.4970
2m 25s (- 1m 36s) (1500 60%) 1.5175
2m 54s (- 1m 14s) (1750 70%) 1.5037
3m 26s (- 0m 51s) (2000 80%) 1.4854
3m 46s (- 0m 25s) (2250 90%) 1.4875
4m 5s (- 0m 0s) (2500 100%) 1.5063


In [15]:
test=X_test[1]

In [27]:
pred = algo.infer(X_test[:100], input_lang, output_lang)
print(pred)

['t+t+t+t-t', 't*t*t*t*t', 't+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't-t-t-t-t', 't+t+t+t', 't+t+t+t+t', 't+t+t+t', 't-t+t-t-0', 't*t*t-t-t', 't+t+t+t+t', 't*t*t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t-t-t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t-t', 't-t+t-t-t', 't-t*t-t-t', 't-t+t-t-t', 't-t*t-t-t', 't*t+t-t-t', 't+t+t+t+t', 't+t+t+t+0', 't+t+t+t+t', 't+t+t+t+t', 't-t*t-t-t', 't+t+t+t+t', 't-t*t-t-t', 't*t*t*t+t', 't-t+t-t-0', 't+t+t-t-t', 't+t+t+t+t', 't+t+t+t+t', 't-t+t-t-t', 't-t*t-t-t', 't*t+t+t+t', 't-t-t-t-t', 't-t+t-t-t', 't+t+t+t+t', 't+t+t-t-t', 't-t-t-t-t', 't*t*t+t+t', 't-t-t-t-t', 't*t*t*t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't-t*t-t-t', 't+t+t+t+t', 't-t-t-t-t', 't+t+t+t+t', 't+t+t-t-t', 't-t-t-t-t', 't+t+t+t+t', 't+t+t+t', 't-t-t-t-t', 't+t+t+t+t', 't-t-t-t-t', 't+t+t+t+t', 't-t-t+t-t', 't-t+t+t-t', 't+t+t+t+t', 't+t+t+t', 't+t+t+t-0', 't+t+t+t+t', 't+t+t+t+t', 't+t+t+t+t', 't-t-t-t-t', 't+t+t+t

In [28]:
accuracy_score(pred, test[:100])

0.0

In [18]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2500, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)

In [26]:
magn_algo.train(train, input_lang, output_lang)

0m 28s (- 4m 16s) (250 10%) 689.5411
0m 57s (- 3m 50s) (500 20%) 690.3261
1m 22s (- 3m 13s) (750 30%) 632.2151
1m 41s (- 2m 32s) (1000 40%) 664.3531
1m 57s (- 1m 57s) (1250 50%) 668.0902
2m 12s (- 1m 28s) (1500 60%) 638.1254
2m 37s (- 1m 7s) (1750 70%) 686.7845
3m 9s (- 0m 47s) (2000 80%) 680.5296
3m 34s (- 0m 23s) (2250 90%) 650.3012
4m 8s (- 0m 0s) (2500 100%) 712.0950


In [31]:
pred = magn_algo.infer(X_test[:1000], input_lang, output_lang)
#print(pred)

In [32]:
accuracy_score(pred, test[:1000])

0.25